In [615]:
import importlib
import esco_utils as eu
import pandas as pd
import json
import matplotlib.pyplot as plt
import numpy as np

importlib.reload(eu)

API_TOKEN = eu.get_token()
client = eu.Client(API_TOKEN)

client.api.base_api_url = 'http://flask_sdk:6221'

{'data': {'token': '12a1bad0-4cb1-464c-8c8e-344957672bef'}, 'message': 'successful', 'meta': {}}


In [56]:
vac_conn, skill_conn = eu.load_skillLab_DB()

In [ ]:
# tags, labels, pred_stats, user_stats = eu.get_stats(client)

In [614]:
df1 = eu.tag_stats(project_name=project_name,email=email)
df2 = eu.label_stats(project_name=project_name,email=email)
df1.columns, df2.columns

(Index(['total_tags', 'search', 'noisy', 'top_5', 'top_10', 'top_10', 'top_20',
        'top_50'],
       dtype='object'),
 Index(['inserted_at', 'total_tasks'], dtype='object'))

In [128]:
project_name = 'SAU3'
label, stats = eu.get_consensus(project_name)
stats = stats.reset_index()
stats.loc[len(stats),:] = ['sum',''] +  stats[['similar','different']].sum().to_list() 
stats.rename(columns={'email_x': 'annotator 1',
                                            'email_y': 'annotator 2',
                                            'similar': 'similar tags',
                                            'different': 'different tags'
                                           })
stats['cosensus'] = 100*stats.similar/(stats.similar+stats.different)
stats['cosensus'] = stats.cosensus.apply(lambda x: str(int(x)) + '%')
stats

,email_x,email_y,similar,different,cosensus
0,hitl1@humansintheloop.org,hitl2@humansintheloop.org,117.0,22.0,84%
1,hitl1@humansintheloop.org,hitl3@humansintheloop.org,104.0,15.0,87%
2,hitl1@humansintheloop.org,hitl4@humansintheloop.org,52.0,26.0,66%
3,hitl2@humansintheloop.org,hitl3@humansintheloop.org,5.0,10.0,33%
4,hitl2@humansintheloop.org,hitl4@humansintheloop.org,91.0,25.0,78%
5,hitl3@humansintheloop.org,hitl4@humansintheloop.org,15.0,38.0,28%
6,,,384.0,136.0,73%


In [573]:
df = stats.reset_index()
df

,project_name,email_x,email_y,similar,different
0,SAU3,hitl1@humansintheloop.org,hitl2@humansintheloop.org,117,22
1,SAU3,hitl1@humansintheloop.org,hitl3@humansintheloop.org,104,15
2,SAU3,hitl1@humansintheloop.org,hitl4@humansintheloop.org,52,26
3,SAU3,hitl2@humansintheloop.org,hitl3@humansintheloop.org,5,10
4,SAU3,hitl2@humansintheloop.org,hitl4@humansintheloop.org,91,25
5,SAU3,hitl3@humansintheloop.org,hitl4@humansintheloop.org,15,38


In [617]:
all_projs = client.get_all_projects()['data']['projects']
name2id = {prj['project_name']:prj['project_id'] for prj in all_projs}
# name2id

In [660]:
project_id = name2id['SAU3']
IN_PROGRESS = 'in-progress',
PENDING = 'pending',
COMPLETE = 'complete'


In [661]:
response = client.get_all_tasks(project_id=project_id,status=IN_PROGRESS)
response['data']['tasks']

[{'_id': '62019e554ee24bc7d74c765e',
  'created_at': 'Mon, 07 Feb 2022 22:33:57 GMT',
  'labelers': ['61fd20bd0e6dac3eff3d3e29', '61fd20da0e6dac3eff3d3e2a'],
  'status': 'in-progress',
  'task-type': 'esco-text-tagging',
  'total_labels': 2,
  'updated_at': 'Mon, 07 Feb 2022 22:33:57 GMT'},
 {'_id': '62019e554ee24bc7d74c765f',
  'created_at': 'Mon, 07 Feb 2022 22:33:57 GMT',
  'labelers': ['61fd20bd0e6dac3eff3d3e29', '61fd20da0e6dac3eff3d3e2a'],
  'status': 'in-progress',
  'task-type': 'esco-text-tagging',
  'total_labels': 2,
  'updated_at': 'Mon, 07 Feb 2022 22:33:57 GMT'},
 {'_id': '62019e554ee24bc7d74c7660',
  'created_at': 'Mon, 07 Feb 2022 22:33:57 GMT',
  'labelers': ['61fd20bd0e6dac3eff3d3e29', '61fd20da0e6dac3eff3d3e2a'],
  'status': 'in-progress',
  'task-type': 'esco-text-tagging',
  'total_labels': 2,
  'updated_at': 'Mon, 07 Feb 2022 22:33:57 GMT'},
 {'_id': '62019e554ee24bc7d74c7661',
  'created_at': 'Mon, 07 Feb 2022 22:33:57 GMT',
  'labelers': ['61fd20bd0e6dac3eff3d3e

In [662]:
task = respnose['data']['tasks'][0]
task = client.get_task(task_id=task['_id'],project_id=project_id)
task

{'data': {'_id': '62019e554ee24bc7d74c765e',
  'callbacks': [None],
  'created_at': 'Mon, 07 Feb 2022 22:33:57 GMT',
  'items': [{'data': {'description': 'وصف / طبيعة العمل موقع العمل (السادس من اكتوبر - المنطقة الصناعية الرابعة و القاهرة الكبرى) حسب موقع المشروع. الاشراف علي اعمال تشطيبات الوحدات السكنية والمكاتب الادارية واعمال الصيانة المتكاملة بالمبانى الادارية والمصانع. متابعة التوريدات العمومية للعميل من حيث تسليم العميل (اشراف جزئى) وأعمال الحفر والخراسانات (ليس نشطا الشركة الدائم) الدراية الكاملة بجميع بنود التشطيبات المعرفة الجيدة بمهام المشرف الفنى وكيفية التعامل مع العمالة والمقاولين و مهندس المالك حصر جميع الأعمال واستلام كافة الأعمال وتسليم العميل رفع تقارير كاملة حسب متطلبات العمل يوميا او اسبوعيا الخبرة في العمل علي كيفية إزالة كافه المعوقات بالموقع ومتابعة خطة التنفيذ مع المهندس المسئول سيكون مسئول عن تجهيز قائمة بالعدد والادوات اللازمة لبدأ العمل بمصنع او وحدة سكنية وتجهيزها من المخزن او استلامها بالموقع عند توريدها (للعلم مجال عمل الشركة داخل مصانع ومبانى ادارية و وحد

In [653]:
task['_id']

'62019e554ee24bc7d74c765e'

In [663]:
pd.read_csv('data/arabic.csv')

,Unnamed: 0,title,description,source,job-id,ar-chars
0,0,مسؤول الشراكات التقنية,إنشاء وتنمية العلاقات مع أصحاب المصلحة في منشآ...,bayt,4485662,True
1,1,مسؤول مبيعات الشركاء,الترويج لحلول الأعمال المتطورة (أنظمة تخطيط مو...,bayt,4485659,True
2,2,شيف تنفيذي,حن نتطلع إلى توظيف شيف تنفيذي جدير بالثناء لإد...,bayt,4485641,True
3,14,فني إلكترونيات وشكبات,القيام بالإجراءات الاولية لصيانة .إجراء الإصلا...,bayt,4485623,True
4,15,فني إلكترونيات وشكبات,القيام بالإجراءات الاولية لصيانة .إجراء الإصلا...,bayt,4485620,True
...,...,...,...,...,...,...
4123,13042,منفذين اعمال لوجيستية (السادس من اكتوبر/المنطق...,بعض مهام العمل: - مسئول عن استلام الشحنات وتخز...,forsana,235623,True
4124,13043,سكرتيرية ادارية - مدينة نصر - القاهرة,شركة الاندلس للتحاليل الطبية ( معامل الحكمة ) ...,forsana,235622,True
4125,13044,مندوب مبيعات و سائق (الاسكندرية) - أمبروز ومحر...,- خبرة من 3-1 سنوات بـنفس المجال السن اقصى حد ...,forsana,235619,True
4126,13045,سائق (رخصة درجة ثانية) - مدينة العبور - القليوبية,- 3 سنوات خبرة على الأقل - يجيد القراءة والكتا...,forsana,235618,True
